In [5]:
import os
import pandas as pd
import numpy as np
from Bio import SeqIO, SeqRecord
import seaborn as sns
import math
import json
import networkx as nwx
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots as pl_make_subplots

from matplotlib import cm

In [6]:
CMAP = {"taxo":{'Actinomycetota': 'rgb(95, 70, 144)',
 'Bacillota': 'rgb(115, 175, 72)',
 'Cyanobacteria': 'rgb(15, 133, 84)',
 'Fusobacteria': 'rgb(56, 166, 165)',
 'NA': 'rgb(29, 105, 150)',
 'Other': 'rgb(184, 189, 181)',#'rgb(237, 173, 8)',
 'Proteobacteria': 'rgb(225, 124, 5)',
 'Nitrospirae': 'rgb(255, 169, 231)' ,
 'Planctomycetota': 'rgb(244, 244, 130)' ,
 'Spirochaetes': "#e60541"#"rgb(226,141,164)"
}}

In [7]:
GBE = json.load(open("../datas/ccyA_gbe_uniprot_accession.json"))

In [8]:
from tqdm import tqdm

FASTA = "../datas/cobahma-fl-sequences-nr.fasta"

records =  {}
for record in SeqIO.parse(open(FASTA),format="fasta"):
    if record.id not in records:
        records[record.id] = record

length_dict = {i: len(j.seq) for i,j in records.items()}        
        
print("Number of sequence in fasta : {}".format(len(records)))

Number of sequence in fasta : 2227


# PARSE COBAHMA DETECTION RESULTS

In [9]:
probes_beta0_df = pd.read_csv("../datas/Probes_CoBaHMA.tsv",sep="\t",index_col=0)
probes_beta0_df.columns = ['Fasta_Identifiant', 'Beginning_Domain', 'End_Domain',"Iteration_0",
       'Iteration_1', 'Iteration_2', 'Iteration_3', 'Iteration_4',
       'Iteration_5', 'Iteration_6', 'Iteration_7',"For_Next_Iteration","Manual_Verification", 'True_CoBaHMA']
probes_beta0_df = probes_beta0_df[['Fasta_Identifiant', 'Beginning_Domain', 'End_Domain',"Iteration_0",
       'Iteration_1', 'Iteration_2', 'Iteration_3', 'Iteration_4',
       'Iteration_5', 'Iteration_6', 'Iteration_7', 'True_CoBaHMA']]
probes_beta0_df.Beginning_Domain = probes_beta0_df.Beginning_Domain+1
probes_beta0_df.End_Domain = probes_beta0_df.End_Domain+1
probes_beta0_df

,Fasta_Identifiant,Beginning_Domain,End_Domain,Iteration_0,Iteration_1,Iteration_2,Iteration_3,Iteration_4,Iteration_5,Iteration_6,Iteration_7,True_CoBaHMA
Identifiant,,,,,,,,,,,,
Chroococcidiopsis_thermalis_PCC_7203,Chroococcidiopsis_thermalis_PCC_7203_0,1,116,True,True,False,False,False,False,False,False,True
Synechococcus_sp_PCC_6312,Synechococcus_sp_PCC_6312_0,1,116,True,True,False,False,False,False,False,False,True
Thermosynechococcus_elongatus,Thermosynechococcus_elongatus_0,1,105,True,True,False,False,False,False,False,False,True
WP_052289956.1,WP_052289956.1,2,116,True,False,False,False,False,False,False,False,True
Synechococcus_PCC_6716,Synechococcus_PCC_6716,2,105,True,False,False,False,False,False,False,False,True
Synechococcus_PCC_6717,Synechococcus_PCC_6717,2,105,True,False,False,False,False,False,False,False,True
NZ_CP018092.1_prot_WP_099799411.1_2103,NZ_CP018092.1_prot_WP_099799411.1_2103,2,105,True,False,False,False,False,False,False,False,True
Cyanothece_sp_PCC_7425,Cyanothece_sp_PCC_7425,4,82,True,False,False,False,False,False,False,False,True
NZ_AP018202.1_prot_WP_126984835.1_8,NZ_AP018202.1_prot_WP_126984835.1_8,2,105,True,False,False,False,False,False,False,False,True


In [10]:
beta0_df = pd.read_csv("../datas/All_CoBaHMA_Bornes_Corrigees.tsv",sep="\t",index_col=0)
beta0_df = beta0_df[beta0_df.index.str.startswith("Uni")]

beta0_df = beta0_df[['Fasta_Identifiant', 'Beginning_Domain', 'End_Domain', 'Iteration_0',
       'Iteration_1', 'Iteration_2', 'Iteration_3', 'Iteration_4',
       'Iteration_5', 'Iteration_6', 'True_CoBaHMA']]
beta0_df.columns = ['Fasta_Identifiant', 'Beginning_Domain', 'End_Domain',
       'Iteration_1', 'Iteration_2', 'Iteration_3', 'Iteration_4',
       'Iteration_5', 'Iteration_6', 'Iteration_7', 'True_CoBaHMA']

beta0_df = pd.concat([probes_beta0_df,beta0_df])
beta0_df.Iteration_0.fillna(False,inplace=True)

print("No of sequence before parsing : " , len(beta0_df.index.unique()))
print("Number of false positive : ", beta0_df[beta0_df.True_CoBaHMA==False].shape[0])

beta0_false_positive = list(beta0_df[beta0_df.True_CoBaHMA==False].index.unique())

# KEEP ONLY TRUE COBAHMA !!!!!
beta0_df = beta0_df[beta0_df.True_CoBaHMA==True]

beta0_mapping_dict = {}
for sid, row in beta0_df.iterrows():
    if sid not in beta0_mapping_dict:
        beta0_mapping_dict[sid]=[]
    beta0_mapping_dict[sid].append(row.to_dict())
print("No of sequence : " , len(beta0_df.index.unique()))

No of sequence before parsing :  2295
Number of false positive :  68
No of sequence :  2227


In [11]:
with open('../datas/True_CoBaHMA.txt','w') as fh:
    for _ in beta0_df[beta0_df.True_CoBaHMA==True].index.unique():
        fh.write("{}\n".format(_))

In [12]:
cobahma_accession = list(beta0_df.index.unique())

print("Number of sequence with at least one True CoBaHMA : ", len(list(beta0_df.index.unique())))
print("Number of sequence with at least one CoBaHMA : ", len(list(beta0_df.Fasta_Identifiant.unique())))

Number of sequence with at least one True CoBaHMA :  2227
Number of sequence with at least one CoBaHMA :  2279


# REMOVE DUPLICATES FROM FASTA FILE

In [13]:
FASTA = "../datas/cobahma-fl-sequences.fasta"

records =  {}
for record in SeqIO.parse(open(FASTA),format="fasta"):
    if record.id not in records and record.id in list(beta0_df.index.unique()):
        records[record.id] = record

SeqIO.write(records.values(), open("../datas/cobahma-fl-sequences-nr.fasta", "w") , format="fasta")

2227

# PARSE DEEPTMHMM RESULTS

In [14]:
tmdatas= []
tmdict = {}
with open("../datas/deepTMHMM/TMRs.gff3", 'r') as gff3:
    for line in gff3.readlines():
        if line.startswith("//"):
            continue
        elif line.startswith("#") and "Number of predicted TMRs" in line:
            seqid = line.strip().split()[1]
            no_tm = line.strip().split()[-1]
            tmdict[seqid]=int(no_tm)
        elif not line.startswith("#"):
            seqid,annot,start,stop = line.strip().split()
            
            tmdatas.append([seqid, None ,"{}-{}".format(int(start),int(stop)) , "TM [{}]".format(annot), None , "deepTMHMM" ])
        else:
            pass
                    
tm_df= pd.DataFrame(tmdatas)   
tm_df.columns = ["Accession" , "E-Value", "Residue Range", "Desc","Code","Tool"]
tm_df.head()

,Accession,E-Value,Residue Range,Desc,Code,Tool
0,UniRef100_A0A098TJ90,None,1-375,TM [inside],None,deepTMHMM
1,UniRef100_V5V6W9,None,1-341,TM [inside],None,deepTMHMM
2,UniRef100_V5V6W9,None,342-356,TM [TMhelix],None,deepTMHMM
3,UniRef100_V5V6W9,None,357-377,TM [outside],None,deepTMHMM
4,UniRef100_V5V6W9,None,378-395,TM [TMhelix],None,deepTMHMM


# PARSE INTERPROSCAN RESUTLS

# REFORMAT ID IN INTERPROSCAN RESULTS

In [15]:
interprocols="""
    protein_accession
    md5
    seqlen
    analysis_type
    signature_accession
    signature_description
    start_location
    stop_location
    evalue
    status
    date
    IPR
    IPR_desc
    mako-cds-classify-intepro-filename
""".split()
interprodf = pd.read_csv("../datas/interproscan.tsv",sep="\t",header=None)
interprodf.columns=interprocols
interprodf.set_index("protein_accession",inplace=True)
interprodf = interprodf[interprodf.index.isin(cobahma_accession)]
#interprodf

In [16]:
#1) load dataframe
raw_interpro_df = interprodf.copy()
 
interprodf.evalue = [float(i) if i!="-" else None  for i in interprodf.evalue ]
interprodf.evalue.replace("-",None,inplace=True)
interprodf.evalue = interprodf.evalue.astype(float)

def clean_interpro(rawdf , analysis_to_remove = ["ProSitePatterns","Coils","MobiDBLite"],only_IPR=True):
    subsetdf = rawdf[~rawdf.analysis_type.isin(analysis_to_remove)]
    if only_IPR:
        return subsetdf[subsetdf.IPR.str.contains("IPR")]
    return subsetdf

def interpro_evalue_correction(evalue,analysis_type):
    if analysis_type=="ProSiteProfiles":
        # Pour cette db la colonne de la e-value correspond en fait à -log(e-val)
        return 10**(-evalue)
    else:
        return evalue 
        
def correct_full_length_identifier(pid,exclude=GBE):        
    if pid not in GBE:
        if not pid.startswith("UniRef100_"):
            return "UniRef100_{}".format(pid)
    return pid

interprodf = clean_interpro(interprodf)
# Not necessary https://www.ebi.ac.uk/interpro/help/faqs/ -> Why are there no e-values associated with InterPro entries?
#interprodf["evalue_corrected"] = interprodf.apply(lambda x: interpro_evalue_correction(x.evalue,x.analysis_type),axis=1)


# Reformat interproscan dataframe

In [17]:

r_interprodf = interprodf[["evalue","start_location","stop_location","IPR_desc","IPR","analysis_type"]].reset_index()
r_interprodf.columns = ["Accession" , "E-Value", "Start","Stop", "Desc","Code","Tool"]
r_interprodf["Residue Range"] = r_interprodf.apply(lambda x: "{}-{}".format(x.Start,x.Stop),axis=1)
r_interprodf["Tool"] = r_interprodf.apply(lambda x: "interproscan|{}".format(x.Tool),axis=1)
r_interprodf = r_interprodf[["Accession" , "E-Value", "Residue Range", "Desc","Code","Tool"]]
r_interprodf


,Accession,E-Value,Residue Range,Desc,Code,Tool
0,UniRef100_A0A1E7GRD2,9.800000e-14,135-209,"Cation-transporting P-type ATPase, N-terminal",IPR004014,interproscan|SMART
1,UniRef100_A0A1E7GRD2,1.250000e-17,650-752,HAD-like superfamily,IPR036412,interproscan|SUPERFAMILY
2,UniRef100_A0A1E7GRD2,2.350000e-28,247-357,"P-type ATPase, A domain superfamily",IPR008250,interproscan|SUPERFAMILY
3,UniRef100_A0A1E7GRD2,5.100000e-56,135-451,"P-type ATPase, transmembrane domain superfamily",IPR023298,interproscan|SUPERFAMILY
4,UniRef100_A0A1E7GRD2,4.400000e-25,354-481,P-type ATPase,IPR001757,interproscan|TIGRFAM
...,...,...,...,...,...,...
5393,UniRef100_R6TVN0,1.570000e-20,197-291,"P-type ATPase, A domain superfamily",IPR008250,interproscan|SUPERFAMILY
5394,UniRef100_R6TVN0,3.700000e-24,164-408,P-type ATPase,IPR001757,interproscan|TIGRFAM
5395,UniRef100_R6TVN0,3.140000e-15,395-517,"P-type ATPase, cytoplasmic domain N",IPR023299,interproscan|SUPERFAMILY
5396,UniRef100_A0A0D7WU36,9.480000e-07,352-456,"P-type ATPase, cytoplasmic domain N",IPR023299,interproscan|SUPERFAMILY


# MAP FUNCTION TO IPR IN A DICTIONNARY

In [18]:
iprdict = raw_interpro_df[["IPR","IPR_desc"]].set_index("IPR").to_dict()["IPR_desc"]
iprdict_great_fun_cat = {}

import re
for _ , fun in iprdict.items():
        fun = fun.split(",")[0]
        if re.search("p-type",fun.lower()):
            fun = "P-type"
        if re.search("HAD", fun):
            fun = "HAD superfamily"
        if re.search("heavy.metal.associated",fun.lower()):
            fun = "Heavy metal-associated domain"
        if re.search("alpha/beta",fun.lower()):
            fun = "Alpha/Beta hydrolase fold"
        if re.search("phosphatidic.acid.phosphatase", fun.lower()):
            fun = "Phosphatidic acid phosphatase type 2/haloperoxidase"
        if re.search("abc.transporter",fun.lower()):
            fun = "ABC transporter"
        if re.search("PIN.*domain" , fun):
            fun = "PIN domain"
        if re.search("diacylglycerol.*kinase" , fun.lower()):
            fun = "Diacylglycerol kinase"
        if re.search("Protein of unknown function",fun):
            fun = fun.replace("Protein","Domain")
        if re.search("winged.helix",fun.lower()):
            fun = "Winged helix DNA-binding domain"
        if re.search("cation.efflux",fun.lower()):
            fun = "Cation efflux protein"
        if re.search("integrase",fun.lower()):
            fun = "Integrase"
            
        iprdict_great_fun_cat[_] = fun

# PARSE DOMAIN MAPPER RESUTLS

In [19]:
domain_mapper_dfs = []
import glob
for file in glob.glob("../datas/domainMAPPER/*"):
    try:
        domain_mapper_df = pd.read_csv(file,comment="#",sep="\t",header=None)
        domain_mapper_df.columns="Accession	E-Value	Residue Range	Property	Architecture	X-group	T-group	F-group	F-id	-".split("\t")
        domain_mapper_dfs.append(domain_mapper_df)
    except pd.errors.EmptyDataError:
        pass

dmdf = pd.concat(domain_mapper_dfs)
dmdf = dmdf[["Accession","E-Value","Residue Range","Property","Architecture","X-group","T-group","F-group","F-id"]]
dmdf["Tool"]="DomainMapper"
dmdf.columns  = ["Accession","E-Value","Residue Range", "Property","Architecture","X-group","T-group"  , "Desc", "Code", "Tool"]

dmdf.head()




,Accession,E-Value,Residue Range,Property,Architecture,X-group,T-group,Desc,Code,Tool
0,UniRef100_A0A218Q7J5,0.000003,229-263,NaN,a+b two layers,Alpha-beta plaits,"HMA, heavy metal-associated domain",HMA,304.3.1.7,DomainMapper
0,UniRef100_A0A2K8T005,0.000006,49-95,NaN,a+b two layers,dsRBD-like,Bacillus phage protein,EUF08408,330.5.1.2,DomainMapper
0,UniRef100_A0A2T0BQL4,0.000008,35-79,NaN,a+b two layers,Alpha-beta plaits,"HMA, heavy metal-associated domain",HMA,304.3.1.7,DomainMapper
1,UniRef100_A0A1D9FGX0,0.000003,210-300,NaN,a/b three-layered sandwiches,HAD domain-like,HAD-like,Hydrolase_3_1,2006.1.1.42,DomainMapper
2,UniRef100_A0A267MLA1,0.000009,36-82,NaN,a+b two layers,Alpha-beta plaits,"HMA, heavy metal-associated domain",HMA,304.3.1.7,DomainMapper


# PARSE HHBLIT COBAHMA RESULTS

In [20]:
r_beta0_df = beta0_df[["Beginning_Domain","End_Domain"]].copy()
r_beta0_df.index.name = "Accession"
r_beta0_df["Residue Range"] = r_beta0_df.apply(lambda x: "{}-{}".format(x.Beginning_Domain,x.End_Domain),axis=1)
r_beta0_df["E-Value"] = None
r_beta0_df["Desc"] = "CoBaHMA"
r_beta0_df["Code"] = None
r_beta0_df["Tool"] = "HHBLIT|CoBaHMA detection"
r_beta0_df = r_beta0_df.reset_index()
r_beta0_df = r_beta0_df[["Accession","E-Value","Residue Range","Desc","Code","Tool"]]
r_beta0_df.head()

,Accession,E-Value,Residue Range,Desc,Code,Tool
0,Chroococcidiopsis_thermalis_PCC_7203,None,1-116,CoBaHMA,None,HHBLIT|CoBaHMA detection
1,Synechococcus_sp_PCC_6312,None,1-116,CoBaHMA,None,HHBLIT|CoBaHMA detection
2,Thermosynechococcus_elongatus,None,1-105,CoBaHMA,None,HHBLIT|CoBaHMA detection
3,WP_052289956.1,None,2-116,CoBaHMA,None,HHBLIT|CoBaHMA detection
4,Synechococcus_PCC_6716,None,2-105,CoBaHMA,None,HHBLIT|CoBaHMA detection


# PARSE PCALF RESULTS

In [21]:
pcalf_df = pd.read_csv("../pcalf-res/pcalf.features.tsv",sep="\t")[
    "sequence_id feature_start 	feature_end	feature_id	e-value".split()
]
pcalf_df.columns = ["Accession","feature_start","feature_stop","Desc","E-Value"]
pcalf_df["Residue Range"]=pcalf_df.apply(lambda x : "{}-{}".format(
    x.feature_start,x.feature_stop),axis=1 )
pcalf_df["Tool"] = "pcalf"
pcalf_df["Code"] = None
r_pcalf_df = pcalf_df[["Accession","E-Value","Residue Range","Desc","Code","Tool"]]
r_pcalf_df = r_pcalf_df[r_pcalf_df.Desc.isin(["Gly1","Gly2","Gly3","GlyX3"])]
r_pcalf_df[r_pcalf_df.Accession == "MWMJ01000016.1_prot_OUE49294.1_331"]


pcalf_summary_df = pd.read_csv("../pcalf-res/pcalf.summary.tsv",sep="\t",index_col=0)

ccya_map_dict = {}
for i in pcalf_summary_df[
    (pcalf_summary_df.flag=="Calcyanin with known N-ter") | 
    (pcalf_summary_df.flag=="Calcyanin with new N-ter")].index:
    ccya_map_dict[i] = "ccyA+" 

# PARSE HMA2 HMM RESULTS

In [22]:
hmmer_cols = """target_name target_accession tlen query_name accession qlen E-value score
bias # of c-Evalue i-Evalue score bias hmm_from hmm_to ali_from ali_to env_from env_to acc""".split()

with open("../datas/HMA2/hma2_per_dom_hit_table.txt") as fh:
    d = []
    for line in fh.readlines():
        if line.startswith("#"):
            continue
        d.append(line.split())
        
hma2_df = pd.DataFrame(d)

hma2_df = hma2_df[list(range(0,22))]
hma2_df.columns = hmmer_cols
hma2_df = hma2_df[["target_name","query_name","E-value","env_from","env_to"]]
hma2_df["E-value"] = hma2_df["E-value"].astype(float)
hma2_df = hma2_df[hma2_df["E-value"] <= 1e-10]
hma2_df["Residue Range"] = hma2_df.apply(lambda x : "{}-{}".format(
    x.env_from,x.env_to),axis=1 )
hma2_df["Code"] = "pfam19991"
hma2_df["Tool"] = "HMMER"
hma2_df["Desc"] = "HMA_2"

hma2_df.columns = ["Accession","query","E-Value","env_from","env_to","Residue Range","Code","Tool","Desc"]
r_hma2_df = hma2_df[["Accession","E-Value","Residue Range","Desc","Code","Tool"]]
r_hma2_df

,Accession,E-Value,Residue Range,Desc,Code,Tool
0,UniRef100_A0A1E8EYI9,3.800000e-72,1-184,HMA_2,pfam19991,HMMER
1,UniRef100_A0A1M4XAC1,7.400000e-71,6-188,HMA_2,pfam19991,HMMER
2,UniRef100_A0A173Y789,1.800000e-70,7-188,HMA_2,pfam19991,HMMER
3,UniRef100_A0A7T0GK02,3.500000e-70,7-188,HMA_2,pfam19991,HMMER
4,UniRef100_B9E0R2,5.000000e-70,31-214,HMA_2,pfam19991,HMMER
...,...,...,...,...,...,...
959,UniRef100_A0A4R1BET3,9.700000e-11,2-86,HMA_2,pfam19991,HMMER
960,UniRef100_A0A1E5L549,9.900000e-11,1-77,HMA_2,pfam19991,HMMER
961,UniRef100_A0A7C9JB03,1.000000e-10,1-92,HMA_2,pfam19991,HMMER
962,UniRef100_A0A0U1NTS5,1.000000e-10,3-78,HMA_2,pfam19991,HMMER


# PARSE TAXONOMY

```bash
python3 retrieve_taxonomy.py -i datas/accession.txt -o datas/taxid.txt;

cut -f 4 datas/taxid.txt  | taxonkit lineage | taxonkit reformat -P > datas/taxonomy.tsv
```

In [23]:
taxodf = pd.read_csv("../datas/taxid.csv",sep="\t",index_col=0)
taxid_mapping_rank = pd.read_csv("../datas/taxonomy.tsv",sep="\t",header=0,index_col=0)
taxid_mapping_rank.columns=["_","ranks"]
taxid_mapping_rank = taxid_mapping_rank["ranks"].to_dict()
taxid_mapping_rank
taxodf["lineage_taxonkit"] = taxodf.taxid.map(taxid_mapping_rank)
taxodf.fillna("",inplace=True)

def parse_taxonkit_lineage(lineage):
    if lineage:
        d = {}
        for r in lineage.split(";"):
            r = r.split("__")
            d[r[0]] = r[-1]
        return d
    return d

def parse_taxo(df):
    d = {}
    if "lineage_taxonkit" in df.columns:
        for i in df.apply(lambda x: {x.name:parse_taxonkit_lineage(x.lineage_taxonkit)} if x.lineage_taxonkit else {x.name:None}, axis=1 ).values:
            d.update(i)
    return d

ranks = pd.DataFrame(parse_taxo(taxodf)).T

taxodf = pd.concat([taxodf,ranks],axis=1)
taxodf.fillna("NA",inplace=True)
taxodf.replace("","NA",inplace=True)


def put_tax_in_other_group(df,col="p",seq_threshold=25):
    df = df.reset_index().set_index(col)
    mapd = {}
    for i in df.index.unique():
        if i not in mapd:
            mapd[i]=""
        no_seq = df.loc[i].shape[0]
        if no_seq > seq_threshold:
            mapd[i] = i
        else:
            mapd[i] = "Other"
    return df.index.map(mapd)

taxodf["p_r"] = put_tax_in_other_group(taxodf,'p',20)

taxodf.to_csv("../datas/taxonomy.reformat.tsv",sep="\t",header=True,index=True) 
taxodf.head()

,seqid,db,taxid,lineage,lineage_taxonkit,k,p,c,o,f,g,s,p_r
WP_052289956.1,NA,NA,1245922.0,NA,k__Bacteria;p__Cyanobacteria;c__;o__Nostocales...,Bacteria,Cyanobacteria,NA,Nostocales,Scytonemataceae,Scytonema,Scytonema millei,Cyanobacteria
Synechococcus_PCC_6716,NA,NA,32048.0,NA,k__Bacteria;p__Cyanobacteria;c__;o__Synechococ...,Bacteria,Cyanobacteria,NA,Synechococcales,Synechococcaceae,Synechococcus,Synechococcus sp. PCC 6716,Cyanobacteria
Synechococcus_PCC_6717,NA,NA,115741.0,NA,k__Bacteria;p__Cyanobacteria;c__;o__Synechococ...,Bacteria,Cyanobacteria,NA,Synechococcales,Synechococcaceae,Synechococcus,Synechococcus sp. PCC 6717,Cyanobacteria
NZ_CP018092.1_prot_WP_099799411.1_2103,NA,NA,33070.0,NA,k__Bacteria;p__Cyanobacteria;c__;o__Thermostic...,Bacteria,Cyanobacteria,NA,Thermostichales,Thermostichaceae,Thermostichus,Thermostichus lividus,Cyanobacteria
Cyanothece_sp_PCC_7425,NA,NA,395961.0,NA,k__Bacteria;p__Cyanobacteria;c__;o__Oscillator...,Bacteria,Cyanobacteria,NA,Oscillatoriales,Cyanothecaceae,Cyanothece,Cyanothece sp. PCC 7425,Cyanobacteria


In [24]:
def get_taxo_mapping_dict(taxodf,col='p_r'):
    return taxodf[col].T.to_dict()

taxodict =  get_taxo_mapping_dict(taxodf)

In [25]:
with open("../datas/other.taxid.txt",'w') as stream:
    stream.write(", ".join([str(int(i)) for i in list(taxodf[taxodf.p_r=="Other"].taxid.unique())]))

In [ ]:
taxodf[taxodf.k!="Bacteria"][["k","p","c","o","f","g","s"]].to_csv("../datas/Sequence_outside_Bacteria_domain.tsv",
                                                                  sep="\t",header=True,
                                                                  index=True)

# PARSE HHBLIT ITER

In [26]:

#fig.show()beta0_df
#seq first_iteration taxo then groupby iteration
d=[]
for rid,row in beta0_df.iterrows():
    ite = 0
    end_ite = 8
    while ite !=  end_ite:        
        col = "Iteration_{}".format(ite)
        if row[col]:
            taxo = None
            d.append([rid, ite])
            break
        ite+=1
tdf = pd.DataFrame(d)
tdf.columns = ["seqid","iteration"]
tdf["phylum_r"]=tdf.seqid.map(taxodict)
tdf = tdf.groupby(["iteration","phylum_r"]).count().reset_index()
domain_dict = taxodf.set_index("p").k.to_dict()

# PARSE NETWORK ANALYSIS RESULTS

In [28]:
NETWORKDIR = '../datas/network_algo_corrected/amp_max_50'
FIGURESDIR = NETWORKDIR + '/Figures'
os.makedirs(FIGURESDIR,exist_ok=True)

# reformat community table

In [218]:
community_table = pd.read_csv(os.path.join( NETWORKDIR,"community_table.tsv" ),sep="\t",index_col=0 )
community_table['prefix'] = community_table.apply(lambda x : x.name.split('_')[0].replace('_',''),axis=1 )
community_table['_commu'] = community_table.apply(lambda x : x.name.split('_')[1].replace('C',''), axis=1 )


potential_ids = []
for i in range(0,community_table._commu.astype(int).max()):
    if i not in community_table._commu.astype(int).tolist():
        potential_ids.append(i)
        
conflict_labels = []

for i in community_table.index.unique().to_list():
    if len(i.split('_'))>2:
        conflict_labels.append(i.split('_')[1].replace('C',''))

resolve_conflict_map = {}
for commu, commudf in community_table[community_table._commu.isin(set(conflict_labels))].groupby('_commu'):
    commudf.sort_values('Community Size',inplace=True, ascending=False )
    keep_as_it = commudf.iloc[0].name 
    for c,crow in commudf.iterrows():
        if c != keep_as_it:
            community_table.at[c,'_commu'] = potential_ids.pop(0)

community_table['community_track'] = community_table.index.tolist()
community_table.index = 'C'+community_table._commu.astype(str)
community_table.index.name = 'Community'   

community_table.to_csv(
    NETWORKDIR+'/community_table_ref.tsv',sep='\t',header=True,index=True)

community_label_map_dict = community_table.reset_index().set_index('community_track').Community.to_dict()
json.dump(community_label_map_dict,open(NETWORKDIR+'/community_map_corrected_labels.json','w'))

In [219]:
# SEE cobahma-network-clean notebook
community_file = os.path.join( NETWORKDIR,"r_communities.json" )
communities = json.load(open(community_file))
communities = {community_label_map_dict[i]:j for i,j in communities.items()}

community_map_dict = {}
for community, seq in communities.items():
    for s in seq:
        community_map_dict[s]=community
        
mmseqs_clu_mapping_dict = json.load(open("../datas/clustering_map_dict.json"))

community_rep_map_dict = community_table["Community Rep"].to_dict()
community_size_map_dict = community_table["Community Size"].to_dict()


### check communty with false positive

In [220]:
for i in beta0_false_positive:
    if i in community_map_dict:
        print(i, community_map_dict[i])



# ANNOTATION DATAFRAMES
*Make two table, one that store sequences's hits and the other that store the percentage of sequence with a specific hit per community*

In [221]:
annot_df = pd.concat([
    dmdf,
    r_interprodf,
])

# FILTER ON E-VALUE : 
threshold = 1e-10
# keep also hits with NaN E-value  (CoBaHMA + deepTMHMM hits)
annot_df = annot_df[ (annot_df['E-Value'].isna()) | (annot_df['E-Value'] <= threshold ) ]

annot_df = pd.concat([
    annot_df,
    r_beta0_df,
    tm_df,
    r_pcalf_df,
    r_hma2_df
])

annot_df.set_index("Accession", inplace=True)
annot_df["Community"] = annot_df.index.map(community_map_dict)
#annot_df["Community NEW"] = annot_df.Community.map(new_communities_map_dict)
annot_df = annot_df[~annot_df.index.isin(beta0_false_positive)] # remove false positive !
annot_df

,E-Value,Residue Range,Property,Architecture,X-group,T-group,Desc,Code,Tool,Community
Accession,,,,,,,,,,
UniRef100_A0A2U0AB95,7.440000e-73,"113-220,743-988",NC,alpha bundles,Calcium ATPase transmembrane domain-related,Calcium ATPase transmembrane domain M,Cation_ATPase_C,5073.1.1.14,DomainMapper,C223
UniRef100_A0A2U0AB95,3.000000e-33,225-334,IS,beta sandwiches,jelly-roll,"Calcium ATPase, transduction domain A",E1-E2_ATPase_N,10.13.1.2,DomainMapper,C223
UniRef100_A0A2U0AB95,6.900000e-47,"431-444,633-778",IS NC,a/b three-layered sandwiches,HAD domain-like,HAD-like,Hydrolase_3_1,2006.1.1.42,DomainMapper,C223
UniRef100_A0A2U0AB95,7.600000e-36,445-634,IS,a+b complex topology,"Metal cation-transporting ATPase, ATP-binding ...","Metal cation-transporting ATPase, ATP-binding ...",Cation_ATPase,267.1.1.3,DomainMapper,C223
UniRef100_A0A257N2D6,3.000000e-15,109-170,NaN,a+b two layers,Alpha-beta plaits,"HMA, heavy metal-associated domain",HMA,304.3.1.7,DomainMapper,C729
...,...,...,...,...,...,...,...,...,...,...
UniRef100_A0A4R1BET3,9.700000e-11,2-86,NaN,NaN,NaN,NaN,HMA_2,pfam19991,HMMER,C437
UniRef100_A0A1E5L549,9.900000e-11,1-77,NaN,NaN,NaN,NaN,HMA_2,pfam19991,HMMER,C4
UniRef100_A0A7C9JB03,1.000000e-10,1-92,NaN,NaN,NaN,NaN,HMA_2,pfam19991,HMMER,C226


In [222]:
# Remove sequence without community, reason : CoBaHMA false positive
annot_df = annot_df[~annot_df.Community.isna()]

#community length and rep sequence 
annot_df["Community Size"] = annot_df.apply(
    lambda x: len(communities[x['Community']]), axis=1)

annot_df["Community Rep"] = annot_df.Community.map(community_rep_map_dict)
print("Number of seq in community >= 5 : ", len(annot_df[annot_df["Community Size"] >= 5].index.unique()))
print("Number  community >= 5 : ", len(annot_df[annot_df["Community Size"] >= 5].Community.unique()))

annot_df["E-Value"] = annot_df["E-Value"].astype(str)
annot_df["Seq length (aa)"] =  annot_df.index.map(length_dict)
#annot_df.Code = t.Code.fillna(0)
annot_df.fillna("NA",inplace=True)
annot_df["Taxonomy"] = annot_df.index.map(taxodict)
annot_df["Taxonomy detail"] = annot_df.index.map(taxodf.p.to_dict())
annot_df.head()

Number of seq in community >= 5 :  1684
Number  community >= 5 :  74


,E-Value,Residue Range,Property,Architecture,X-group,T-group,Desc,Code,Tool,Community,Community Size,Community Rep,Seq length (aa),Taxonomy,Taxonomy detail
Accession,,,,,,,,,,,,,,,
UniRef100_A0A2U0AB95,7.440000000000001e-73,"113-220,743-988",NC,alpha bundles,Calcium ATPase transmembrane domain-related,Calcium ATPase transmembrane domain M,Cation_ATPase_C,5073.1.1.14,DomainMapper,C223,1,UniRef100_A0A2U0AB95,1000,Proteobacteria,Proteobacteria
UniRef100_A0A2U0AB95,3e-33,225-334,IS,beta sandwiches,jelly-roll,"Calcium ATPase, transduction domain A",E1-E2_ATPase_N,10.13.1.2,DomainMapper,C223,1,UniRef100_A0A2U0AB95,1000,Proteobacteria,Proteobacteria
UniRef100_A0A2U0AB95,6.9e-47,"431-444,633-778",IS NC,a/b three-layered sandwiches,HAD domain-like,HAD-like,Hydrolase_3_1,2006.1.1.42,DomainMapper,C223,1,UniRef100_A0A2U0AB95,1000,Proteobacteria,Proteobacteria
UniRef100_A0A2U0AB95,7.600000000000001e-36,445-634,IS,a+b complex topology,"Metal cation-transporting ATPase, ATP-binding ...","Metal cation-transporting ATPase, ATP-binding ...",Cation_ATPase,267.1.1.3,DomainMapper,C223,1,UniRef100_A0A2U0AB95,1000,Proteobacteria,Proteobacteria
UniRef100_A0A257N2D6,3e-15,109-170,NA,a+b two layers,Alpha-beta plaits,"HMA, heavy metal-associated domain",HMA,304.3.1.7,DomainMapper,C729,1,UniRef100_A0A257N2D6,500,Proteobacteria,Proteobacteria


In [223]:
annotation_file = os.path.join(NETWORKDIR , "cobahma_clean_annotation_with-taxo.tsv")
annot_df.to_csv(annotation_file,sep="\t",header=True,index=True)

In [224]:
# Remove duplicates
t= annot_df.reset_index().set_index(["Accession","Desc"])
t = t[~t.index.duplicated(keep="first")].reset_index()

t[t["Community Size"] >= 5].groupby(["Community","Desc","Code"]).count()
annot_df_per_commu = t[t["Community Size"] >= 5].groupby(
    ["Community","Desc","Code"]).count()["Residue Range"].reset_index()
annot_df_per_commu
annot_df_per_commu.columns = ["Community","Desc","Code","No sequences"]
annot_df_per_commu["Community Size"] = annot_df_per_commu.Community.map(
    annot_df.set_index("Community")["Community Size"].to_dict() )
annot_df_per_commu["%Annotated Seq"] = round(annot_df_per_commu["No sequences"]/annot_df_per_commu["Community Size"] * 100,1)
annot_df_per_commu.set_index("Community",inplace=True)
annot_df_per_commu
#annot_df_per_commu.to_csv("../datas/community_gt5_seq_proportion_by_annotation.tsv" , sep="\t",header=True,index=True)


,Desc,Code,No sequences,Community Size,%Annotated Seq
Community,,,,,
C106,CoBaHMA,NA,9,9,100.0
C106,TM [inside],NA,9,9,100.0
C140,CoBaHMA,NA,19,19,100.0
C140,E1-E2_ATPase_N,10.13.1.2,19,19,100.0
C140,HAD superfamily,IPR023214,19,19,100.0
...,...,...,...,...,...
C98,"P-type ATPase, haloacid dehalogenase domain",IPR044492,13,13,100.0
C98,"P-type ATPase, transmembrane domain superfamily",IPR023298,13,13,100.0
C98,TM [TMhelix],NA,13,13,100.0



# BARPLOT ITERATION/TAXO

In [225]:
beta0_df["taxo"] = beta0_df.index.map(taxodict)
beta0_df["p"] = beta0_df.index.map(taxodf.p.to_dict())
beta0_df["k"] = beta0_df.index.map(taxodf.k.to_dict())

In [226]:
chordeae = "UniRef100_A0A401TJW7"
beta0_df.loc[chordeae] # -> Found in first transitive search

Fasta_Identifiant    UniRef100_A0A401TJW7_0
Beginning_Domain                          1
End_Domain                               83
Iteration_0                           False
Iteration_1                            True
Iteration_2                            True
Iteration_3                            True
Iteration_4                            True
Iteration_5                            True
Iteration_6                            True
Iteration_7                            True
True_CoBaHMA                           True
taxo                                  Other
p                                  Chordata
k                                 Eukaryota
Name: UniRef100_A0A401TJW7, dtype: object

In [227]:
beta0_df[beta0_df["k"] == "Archaea"]

,Fasta_Identifiant,Beginning_Domain,End_Domain,Iteration_0,Iteration_1,Iteration_2,Iteration_3,Iteration_4,Iteration_5,Iteration_6,Iteration_7,True_CoBaHMA,taxo,p,k
Identifiant,,,,,,,,,,,,,,,
UniRef100_A0A662RLZ9,UniRef100_A0A662RLZ9_0,1,76,False,True,True,True,True,True,True,True,True,Other,Euryarchaeota,Archaea
UniRef100_A0A843H6L9,UniRef100_A0A843H6L9_0,1,74,False,True,True,True,True,True,True,True,True,Other,Euryarchaeota,Archaea
UniRef100_A0A662NFT6,UniRef100_A0A662NFT6_0,1,77,False,True,True,True,True,True,True,True,True,Other,Euryarchaeota,Archaea
UniRef100_A0A843EZ90,UniRef100_A0A843EZ90_0,1,78,False,True,True,True,True,False,False,False,True,Other,Euryarchaeota,Archaea


In [228]:
taxodf = taxodf[taxodf.index.isin(beta0_df.index)]

In [229]:
tdf=tdf.sort_values("iteration")
tdf['plotly_text'] = tdf.apply(lambda x: str(x.seqid) if x.seqid > 50 else "", axis=1)

In [230]:
tdsum = tdf.groupby('iteration').sum()
tdsum.columns=['count']

/var/folders/zq/dyg14bbs529d4mgyfl466ngm0000gn/T/ipykernel_99607/2769886546.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [231]:

fig = px.bar(tdf, x="iteration", y="seqid", color="phylum_r",
             #title="Number of sequences recruited at each iteration in regard to their taxonomy ",
#             color_discrete_sequence=colors ,
             color_discrete_map=CMAP["taxo"],
             text=tdf['plotly_text'],#opacity=0.5,
             labels={
                     "iteration": "Number of transitive searchs",
                     "seqid": "Number of sequences",
                     "phylum_r": "Taxonomy [Phylum]"
                 },)

fig.add_trace(go.Scatter(
    x=tdsum.index, 
    y=tdsum['count'],
    text=tdsum['count'],
    mode='text',
    textposition='top center',
    textfont=dict(
        size=18,
        color="crimson"
    ),
    showlegend=False
))

annotations = []
for i,j in taxodf[taxodf.p_r=="Other"].groupby("p").count().iterrows():
    if i in domain_dict and domain_dict[i] == "Eukaryota":
        annotations.append("<b>{} [{}]*</b>".format(i,j.seqid))
    elif i in domain_dict and domain_dict[i] == "Archaea":
        annotations.append("<i>{} [{}]*</i>".format(i,j.seqid))
    else:
        annotations.append("{} [{}]".format(i,j.seqid))
txt_annotations = "<b>Other:</b><br>"+"<br>".join(annotations)
fig.add_annotation(text=txt_annotations, 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.4,
                    y=1,
                    #bordercolor='black',
                    #borderwidth=1
                  )

fig.update_traces(textfont_size=12)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',margin={"r":600},
    plot_bgcolor='rgba(0,0,0,0)',width=1900,height=800)

fig.write_html(FIGURESDIR  + "/Supplementary_Data_2.html")
fig.write_image(FIGURESDIR + "/Supplementary_Data_2.svg")
fig.write_image(FIGURESDIR + "/Supplementary_Data_2.png")
fig.write_image(FIGURESDIR + "/Supplementary_Data_2.pdf")
fig.show()

# Community single cobahma

In [232]:
cobahma_only_commu = ["C437","C201","C583","C675","C399","C397","C71","C685","C688"]

# Make community taxo composition chart

In [233]:
datas = []
tmp_taxo_dict = taxodict.copy()#taxodf.p.to_dict()
for cid, seq in communities.items():
    for s in seq:
        if s in tmp_taxo_dict:
            taxo = tmp_taxo_dict[s]
        else:
            taxo = "NA"
        datas.append([cid,taxo,s])
            
del tmp_taxo_dict            

community_taxo_count = pd.DataFrame(datas).groupby([0,1]).count().reset_index()
community_taxo_count.columns=["Community","Phylum","Count"]
community_taxo_count["Community Size"] = community_taxo_count.Community.map(community_size_map_dict)
community_taxo_count["Proportion"] = community_taxo_count["Count"]/community_taxo_count["Community Size"]*100
community_taxo_count = community_taxo_count[community_taxo_count["Community Size"]>=5]
community_taxo_count.sort_values(["Proportion"],inplace=True,ascending=False)


In [234]:
tmp = community_taxo_count.sort_values("Proportion").set_index("Community")
community_taxo_order = list(tmp[~tmp.index.duplicated(keep="last")].reset_index().sort_values(
    ["Phylum","Proportion"]).Community)
del tmp
taxo_order_file = os.path.join(NETWORKDIR,'community_taxo_order.clean.txt')
with open(taxo_order_file,"w") as f:
    for c in community_taxo_order:
        f.write(c+"\n")

In [235]:
no_bacterial_seq = list(taxodf[~taxodf.k.isin(['NA','Bacteria','Chordata'])].index.unique())
# find the archeae ...
for i,j in communities.items():
    if len(j) >= 5:
        for _ in j:
            if _ in no_bacterial_seq:
                print(i)


C366


In [236]:
taxo_for_community_part = taxodf[taxodf.index.isin(annot_df.index)]
print(taxo_for_community_part.shape)
taxo_for_community_part = taxo_for_community_part[taxo_for_community_part.index.isin(annot_df[annot_df["Community Size"]>=5].index)]

(2227, 13)


In [244]:

phylum_in_legend=[]
data= []
print('start')
for i in range(len(community_taxo_order)):
    community = community_taxo_order[i]    
    cdf = community_taxo_count[community_taxo_count.Community==community].copy()

    commu = community
    
    cdf.sort_values("Proportion",inplace=True,ascending=False)
    cdf.set_index("Phylum",inplace=True)
    for phylum, row in cdf.iterrows():
        data.append(go.Bar(y=[commu],
                           x=[row.Proportion],
                           orientation='h',
                           name=phylum,
                           text="{}/{}".format(row.Count,row["Community Size"]),
                           legendgroup=phylum,
                           marker=dict(color= CMAP["taxo"][phylum] if phylum in CMAP["taxo"] else "pink" ),
                           showlegend= phylum not in phylum_in_legend)
                   ) # show the legend only once for each column 
        if phylum not in phylum_in_legend:
            phylum_in_legend.append(phylum)
            

# figure
fig = go.Figure(data=data)

annotations = []
for i,j in taxo_for_community_part[taxo_for_community_part.p_r=="Other"].groupby("p").count().iterrows():
    if i in domain_dict and domain_dict[i] == "Eukaryota":
        annotations.append("<b>{} [{}]*</b>".format(i,j.seqid))
    elif i in domain_dict and domain_dict[i] == "Archaea":
        annotations.append("<i>{} [{}]*</i>".format(i,j.seqid))
    else:
        annotations.append("{} [{}]".format(i,j.seqid))

txt_annotations = "<b>Other:</b><br>"+"<br>".join(annotations)



format_ytick_dict={}
for i in community_taxo_order:
    if i in cobahma_only_commu:
        format_ytick_dict[i] = "<b>"+i+"</b>"
    elif i == "C366":
        format_ytick_dict[i] = i+"*"
    else:
        format_ytick_dict[i] = i

fig.update_traces(textfont_size=12)
fig.update_layout(
    barmode='stack',
    paper_bgcolor='rgba(0,0,0,0)',margin={"r":250},
    plot_bgcolor='rgba(0,0,0,0)',width=1000,height=2000,
    yaxis={
      "title":"Community",
      "categoryarray":community_taxo_order,
      "categoryorder":"array",
      "tickvals":community_taxo_order,
      "ticktext":[format_ytick_dict[i] for i in community_taxo_order]
      },
    xaxis={
        "title":"% of sequences ",
        "categoryorder":"total descending"
      },
    legend=dict(
        title="Taxonomy [Phylum]"
    )
)

fig.add_annotation(text=txt_annotations, 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    y=0.8,
                    x=1.35,
                  )


fig.write_html(FIGURESDIR + "/Figure_4_big_community_taxonomy.html")
fig.write_image(FIGURESDIR +"/Figure_4_big_community_taxonomy.pdf")
fig.write_image(FIGURESDIR + "/Figure_4_big_community_taxonomy.svg")
fig.show()


start


# Make OM comunity composition chart (Interproscan only ?)
*need to group IPRs into several modular orga*
- CoBaHMA-0
- CoBaHMA-1,2
- CoBaHMA-1,2,3
- CoBaHMA,ABC
- CoBaHMA,P-TYPE
- CoBaHMA,phosphatase

In [245]:
if "Accession" in annot_df.columns:
    annot_df.set_index("Accession",inplace=True)
annot_df["start"] = annot_df.apply(lambda x : x["Residue Range"].split("-")[0],axis=1)
annot_df.sort_values("start",inplace=True)

tmp = {}
cobahma_map_dict = {}
for i, row in annot_df.reset_index().iterrows():
    seq = row.Accession
    if row.Desc == "CoBaHMA":
        if seq not in tmp:
            tmp[seq] = -1
        tmp[seq]+=1
        cobahma_map_dict[i] = "CoBaHMA_{}".format(tmp[seq])
annot_df   

,E-Value,Residue Range,Property,Architecture,X-group,T-group,Desc,Code,Tool,Community,Community Size,Community Rep,Seq length (aa),Taxonomy,Taxonomy detail,start
Accession,,,,,,,,,,,,,,,,
UniRef100_A0A1I0SSG2,1e-10,1-88,NA,NA,NA,NA,HMA_2,pfam19991,HMMER,C444,101,UniRef100_A0A522V264,127,Proteobacteria,Proteobacteria,1
UniRef100_A0A0A5GGF7,6.3e-12,1-78,NA,NA,NA,NA,HMA_2,pfam19991,HMMER,C4,18,UniRef100_A0A1H0C0W7,190,Bacillota,Bacillota,1
UniRef100_A0A6G9SFK5,6.1e-12,1-143,NA,NA,NA,NA,HMA_2,pfam19991,HMMER,C341,23,UniRef100_A0A6M0J010,201,Cyanobacteria,Cyanobacteria,1
UniRef100_B9E0R3,5.8e-12,1-126,NA,NA,NA,NA,HMA_2,pfam19991,HMMER,C685,64,UniRef100_A0A371IS94,126,Bacillota,Bacillota,1
UniRef100_A0A538R1S0,5.3e-12,1-163,NA,NA,NA,NA,HMA_2,pfam19991,HMMER,C294,7,UniRef100_A0A2V7B4L6,169,Other,Candidatus Rokubacteria,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UniRef100_A0A2T5JWD8,nan,998-1014,NA,NA,NA,NA,TM [outside],NA,deepTMHMM,C98,13,UniRef100_A0A0M0SKF3,1059,Proteobacteria,Proteobacteria,998
UniRef100_A0A2L2XEP1,nan,998-1010,NA,NA,NA,NA,TM [outside],NA,deepTMHMM,C46,1,UniRef100_A0A2L2XEP1,1110,Bacillota,Bacillota,998
UniRef100_A0A2U3JXS2,nan,998-1018,NA,NA,NA,NA,TM [TMhelix],NA,deepTMHMM,C556,23,UniRef100_A0A7C4R520,1026,Proteobacteria,Proteobacteria,998


In [246]:
annot_df["Desc_copy"] = annot_df.Desc
annot_df.reset_index(inplace=True)
annot_df.Desc_copy = annot_df.reset_index().apply(lambda x: cobahma_map_dict[x.name] if x.name in cobahma_map_dict else x.Desc , axis=1 )


In [247]:
simplified_annotation_map_dict = {}
for _ , row in annot_df[annot_df["Community Size"]>=5][["Desc","Code"]].iterrows():
    if "p-type" in row.Desc.lower() or "had" in row.Desc.lower():
        d = "P-type"
    elif "phosphatase" in row.Desc.lower():
        d = "Phosphatase"
    elif "abc" in row.Desc.lower():
        d = "ABC-transporter"
    elif "heavy metal-associated" in row.Desc.lower():
        d = "HMA"
    elif  "glyx3" in row.Desc.lower():
        d = "GlyX3"
    elif "p-loop" in row.Desc.lower():
        d = "P-loop"
    else:
        d = row.Desc 

    simplified_annotation_map_dict[row.Desc]   = d    


In [248]:
ignore_annot = ["Gly1","Gly2","Gly3","TM [inside]", "TM [outside]", "TM [TMhelix]", "HMA_2","Type 1 protein exporter" ]
tmp = annot_df[~annot_df.Desc.isin(ignore_annot)]
tmp = tmp[tmp.Tool!="DomainMapper"]
modorg_datas = []
for seq, seq_annots in tmp.reset_index().groupby("Accession"):
    om = []
    seq_annots.sort_values("start",inplace=True)
    for e in seq_annots.Desc_copy:
        if e in simplified_annotation_map_dict:
            e = simplified_annotation_map_dict[e]
        om.append(e)
    modorg_datas.append(
        (seq,";".join(sorted(list(set(om)))))
    )
modorg_df = pd.DataFrame(modorg_datas)
modorg_df.columns = ["Accession","Mod Org"]
modorg_df["Community"] =  modorg_df.Accession.map(community_map_dict)
#modorg_df["Community Size"] = modorg_df.Community.map(community_size_map_dict)
community_modorg_count = modorg_df.groupby(["Community","Mod Org"]).count().reset_index()
community_modorg_count.columns = ["Community","Mod Org","Count"]
community_modorg_count["Community Size"] = community_modorg_count.Community.map(community_size_map_dict)
community_modorg_count["Proportion"] = community_modorg_count["Count"]/community_modorg_count["Community Size"]*100
community_modorg_count = community_modorg_count[community_modorg_count["Community Size"]>=5]
community_modorg_count

,Community,Mod Org,Count,Community Size,Proportion
7,C106,CoBaHMA_0,9,9,100.0
27,C140,CoBaHMA_0;P-type,19,19,100.0
28,C141,CoBaHMA_0,46,46,100.0
33,C154,CoBaHMA_0;P-type,5,5,100.0
51,C192,CoBaHMA_0;GlyX3,15,15,100.0
...,...,...,...,...,...
425,C740,CoBaHMA_0,10,10,100.0
428,C744,CoBaHMA_0;P-type,6,6,100.0
429,C75,CoBaHMA_0,80,80,100.0
450,C93,CoBaHMA_0,6,6,100.0


In [249]:
modorg_cmap = {'CoBaHMA_0;P-type':"lightcyan",
 'CoBaHMA_0':"#a6c9b6",
 'CoBaHMA_0;GlyX3':"green",
 'CoBaHMA_0;CoBaHMA_1':"#365946",
 'CoBaHMA_0;Phosphatase':"violet",
 'AAA+ ATPase domain;ABC-transporter;CoBaHMA_0;P-loop':"orange",
 'CoBaHMA_0;HMA;P-type':"cyan",
 'CoBaHMA_0;CoBaHMA_1;CoBaHMA_2':'#21596e'}

In [250]:

modorg_in_legend=[]
data= []
for i in range(len(community_taxo_order)):
    community = community_taxo_order[i]
    cdf = community_modorg_count[community_modorg_count.Community==community].copy()
    cdf.sort_values("Proportion",inplace=True,ascending=False)
    cdf.set_index("Mod Org",inplace=True)
    for modorg, row in cdf.iterrows():
        data.append(go.Bar(x=[community],
                           y=[row.Proportion],
                           name=modorg,
                           text="{}/{}".format(row.Count,row["Community Size"]),
                           legendgroup=modorg,
                           marker=dict(color= modorg_cmap[modorg] if modorg in modorg_cmap else "grey" ),
                           showlegend= modorg not in modorg_in_legend)) # show the legend only once for each column 
        if modorg not in modorg_in_legend:
            modorg_in_legend.append(modorg)
            
# layout
layout = dict(barmode='stack',
              yaxis={'title': 'amount'},
              xaxis={'type': 'category', 'title': 'month'})

# figure
fig = go.Figure(data=data)

fig.update_traces(textfont_size=12)
fig.update_layout(
    barmode='stack',
    paper_bgcolor='rgba(0,0,0,0)',margin={"r":600},
    plot_bgcolor='rgba(0,0,0,0)',width=1900,height=800,
    xaxis={
      "title":"Community",
      "categoryarray":community_taxo_order,
      "categoryorder":"array"
       
      },
    yaxis={
        "title":"% of sequences ",
        "categoryorder":"total descending"
  }
)        
fig.write_html(FIGURESDIR + "/not_use_community_modorg_composition.html")
fig.write_image(FIGURESDIR +  "/not_use_community_modorg_composition.pdf")
fig.show()

# Make network annotation file
*Store community , subgraph taxonomy and other datas for each sequence*

In [251]:
a1 = modorg_df.set_index("Accession")['Mod Org']
a2 = pd.DataFrame.from_dict({"Seq Length":length_dict})
a3 = pd.read_csv(NETWORKDIR+"/community_map_table.tsv",sep="\t",index_col=0)

network_annotation_file = NETWORKDIR + "/network_annotation_file.tsv"
pd.concat([a1,a2,a3],axis=1).to_csv(network_annotation_file,sep='\t',header=True,index=True)

# Make community dataframe

In [252]:
community_df = pd.read_csv(NETWORKDIR+"/community_table.tsv",sep="\t",header=0,index_col=0)

In [253]:
d = {}

tmpdf = annot_df[((annot_df.Code.str.startswith("IPR")) & (annot_df.Tool.str.contains("interproscan") )) | 
         (annot_df.Tool.str.contains("DomainMapper"))][["Community","Code","Desc","Tool"]]



for commu , commu_fun_desc in tmpdf.set_index("Community").iterrows():
    if commu not in d:
        d[commu] = {"IPRS":[],"IPRS_Functions":[],"DomainMapper_Ecod":[],"DomainMapper_Functions":[]}

    code_label = "DomainMapper_Ecod"     
    func_label = "DomainMapper_Functions"
    if commu_fun_desc.Code.startswith("IPR"):
        code_label = "IPRS"
        func_label = "IPRS_Functions"
    if commu_fun_desc.Code not in d[commu][code_label]:
        d[commu][code_label].append(commu_fun_desc.Code)
        d[commu][func_label].append(commu_fun_desc.Desc)        
rd = {}
for commu,func in d.items():
    rd[commu] = {"IPRS":[],"IPRS_Functions":[],"DomainMapper_Ecod":[],"DomainMapper_Functions":[]}
    rd[commu]["IPRS_Functions"] = ";".join(func["IPRS_Functions"])
    rd[commu]["IPRS"] = ";".join(func["IPRS"])
    rd[commu]["DomainMapper_Functions"] = ";".join(func["DomainMapper_Functions"])
    rd[commu]["DomainMapper_Ecod"] = ";".join(func["DomainMapper_Ecod"])
community_fun_df = pd.DataFrame(rd).T
community_fun_df.head()

,IPRS,IPRS_Functions,DomainMapper_Ecod,DomainMapper_Functions
C659,IPR002852,Uncharacterised protein family UPF0251,,
C398,IPR009963;IPR001757;IPR008250,Protein of unknown function DUF1490;P-type ATP...,5073.1.2.3;10.13.1.2,E1-E2_ATPase_C;E1-E2_ATPase_N
C525,IPR036163;IPR006121;IPR027256;IPR023298;IPR001...,Heavy metal-associated domain superfamily;Heav...,304.3.1.7;5073.1.2.3;10.13.1.2;2006.1.1.42;267...,HMA;E1-E2_ATPase_C;E1-E2_ATPase_N;Hydrolase_3_...
C42,IPR023298;IPR004014;IPR001757;IPR008250;IPR044...,"P-type ATPase, transmembrane domain superfamil...",5073.1.1.14;10.13.1.2;NA;2006.1.1.42;267.1.1.3,Cation_ATPase_C;E1-E2_ATPase_N;Cation_ATPase_C...
C438,IPR036163;IPR027256;IPR023298;IPR001757;IPR008...,Heavy metal-associated domain superfamily;P-ty...,304.3.1.7;5073.1.2.3;10.13.1.2;2006.1.1.42;267...,HMA;E1-E2_ATPase_C;E1-E2_ATPase_N;Hydrolase_3_...


In [254]:
community_desc_table = pd.concat([community_table,community_fun_df],axis=1).fillna("NA")
community_desc_table.index.name ="Community"
community_desc_table.to_csv(NETWORKDIR + "/community_desc.tsv",sep="\t",header=True,index=True)
print(community_desc_table.shape)
community_desc_table.head()

(439, 17)


,Community Rep,Rep degree,Community Size,subgraph,Rep Degree,mean_seq_len,longest_seqs,longest_len,shortest_seqs,shortest_len,prefix,_commu,community_track,IPRS,IPRS_Functions,DomainMapper_Ecod,DomainMapper_Functions
Community,,,,,,,,,,,,,,,,,
C3,UniRef100_A0A1M3N6F4,0,1,Subgraph_178,0,163.000000,UniRef100_A0A1M3N6F4,163,UniRef100_A0A1M3N6F4,163,us,3,us_C3,NA,NA,NA,NA
C4,UniRef100_A0A1H0C0W7,12,18,Subgraph_17,12,194.111111,UniRef100_A0A6P1HK34,202,UniRef100_A0A0A5GGF7;UniRef100_A0A1L3MRH9;UniR...,190,ubc,4,ubc_C4,NA,NA,NA,NA
C5,UniRef100_A0A662ZLY5,4,5,Subgraph_52,4,218.800000,UniRef100_R5EHL9,222,UniRef100_A0A662ZMY2,210,ubc,5,ubc_C5,NA,NA,NA,NA
C6,UniRef100_A0A0R1VKN6,0,1,Subgraph_179,0,487.000000,UniRef100_A0A0R1VKN6,487,UniRef100_A0A0R1VKN6,487,us,6,us_C6,IPR001757;IPR008250;IPR023214;IPR023299,"P-type ATPase;P-type ATPase, A domain superfam...",5073.1.2.3;10.13.1.2;267.1.1.5,E1-E2_ATPase_C;E1-E2_ATPase_N;P-type_ATPase_Cu...
C7,UniRef100_A0A4Q5L4V0,4,5,Subgraph_53,4,795.800000,UniRef100_A0A5Q0BFQ5,821,UniRef100_A0A858QAA3,786,sc,7,sc_C7,IPR008250;IPR023299;IPR023214;IPR036412,"P-type ATPase, A domain superfamily;P-type ATP...",10.13.1.2;2006.1.1.42,E1-E2_ATPase_N;Hydrolase_3_1


# Alluvial plot taxo / func

In [255]:
great_func_datas = {}

#with open("../datas/sequence_great_func_group.tsv",'w') as f:
for seqid , sdf in annot_df[~annot_df.Tool.isin(["DomainMapper","deepTMHMM","HMMER"])].groupby("Accession"):
    great_func_group = None
    seq_desc = ";".join(list(sdf.Desc_copy.unique())).lower()

    if "glyx3" in seq_desc:
        great_func_group = "GlyX3" 
    elif "phosphatase" in seq_desc:        
        great_func_group = "Phosphatase"     
    elif "abc" in seq_desc:
        great_func_group = "ABC transporter"
    elif "p-type" in seq_desc or "had" in seq_desc:
        great_func_group = "P-type"
    elif "cobahma_2"  in seq_desc:
        great_func_group = "CoBaHMA_X3"
    elif "cobahma_1"  in seq_desc:
        great_func_group = "CoBaHMA_X2"
    else:
        great_func_group = "CoBaHMA_X1"
#    f.write("{}\t{}\t{}\n".format(
#        seqid,
#        great_func_group,
#        community_map_dict[seqid]
#    ))

    if seqid not in great_func_datas:
        great_func_datas[seqid] = {"taxo":None,"great_func_group": great_func_group}
    great_func_datas[seqid]["taxo"]= taxodict[seqid]


In [256]:
sankey_datas = []
for cid,sequences in communities.items():
    if len(sequences)>=5:
        p_type = 0
        phosphatase = 0
        glyx3 = 0
        abc = 0
        cobahmaX1 = 0
        cobahmaX2 = 0
        cobahmaX3 = 0        
        # big CO
        taxo = {}
        for seq in sequences:
            d = great_func_datas[seq]
            # TAXO
            if d["taxo"] not in taxo:
                taxo[d["taxo"]]=0
            taxo[d["taxo"]]+=1
            # FUNC
            if "Phosphatase" in d["great_func_group"]:
                phosphatase+=1
                continue
            elif "P-type" in d["great_func_group"]:
                p_type+=1
                continue
            elif "GlyX3" in d["great_func_group"]:
                glyx3+=1
                continue
            elif "ABC transporter" in d["great_func_group"]:
                abc+=1
                continue
            elif "CoBaHMA_X3" in d["great_func_group"]:
                cobahmaX3+=1
                continue
            elif "CoBaHMA_X2" in d["great_func_group"]:
                cobahmaX2+=1
                continue                
            elif "CoBaHMA_X1" in d["great_func_group"]:
                cobahmaX1+=1
                continue 
        sankey_datas+=[("P-type",cid,p_type),
                   ("Phosphatase",cid,phosphatase),
                   ("ABC transporter",cid,abc),
                   ("GlyX3",cid,glyx3),
                   ("CoBaHMA_X1",cid,cobahmaX1),
                   ("CoBaHMA_X2",cid,cobahmaX2),
                   ("CoBaHMA_X3",cid,cobahmaX3),                   
                  ]
        for taxo,cpt in taxo.items():
            sankey_datas.append((cid,taxo,cpt))
        
sankeydf = pd.DataFrame(sankey_datas)
sankeydf.columns=["src","target","value"]

sankeydf = sankeydf[sankeydf["value"]>0]
sankeydf.fillna("NA",inplace=True)


sankey_idx_map = {}
cpt = 0
for i in set(list(sankeydf.src) + list(sankeydf.target)):
    sankey_idx_map[i] = cpt
    cpt+=1

sankeydf["src_r"] = sankeydf.src.map(sankey_idx_map)
sankeydf["target_r"] = sankeydf.target.map(sankey_idx_map)
sankeydf["label"] = sankeydf.apply(lambda x : list(sankey_idx_map.keys())[x.src_r],axis=1)

In [257]:
cmap_taxo_rgba = {'Actinomycetota': 'rgba(95, 70, 144,0.1)',
 'Bacillota': 'rgba(115, 175, 72,0.1)',
 'Cyanobacteria': 'rgba(15, 133, 84, 1)',
 'Fusobacteria': 'rgba(56, 166, 165,0.1)',
 'NA': 'rgba(29, 105, 150,0.1)',
 'Other': 'rgba(184, 189, 181,0.1)',#'rgb(237, 173, 8)',
 'Proteobacteria': 'rgba(225, 124, 5,0.1)',
 'Nitrospirae': 'rgba(255, 169, 231,0.1)' ,
 'Planctomycetota': 'rgba(244, 244, 130,0.1)' ,
 'Spirochaetes': "rgba(230, 5, 65 , 0.1)"#"rgb(226,141,164)"
}

cmap_taxo_rgba_no_opa = {'Actinomycetota': 'rgba(95, 70, 144,0.7)',
 'Bacillota': 'rgba(115, 175, 72,0.7)',
 'Cyanobacteria': 'rgba(15, 133, 84, 0.7)',
 'Fusobacteria': 'rgba(56, 166, 165,0.7)',
 'NA': 'rgba(29, 105, 150,0.7)',
 'Other': 'rgba(184, 189, 181,0.75)',#'rgb(237, 173, 8)',
 'Proteobacteria': 'rgba(225, 124, 5,0.7)',
 'Nitrospirae': 'rgba(255, 169, 231,0.7)' ,
 'Planctomycetota': 'rgba(244, 244, 130,0.7)' ,
 'Spirochaetes': "rgba(230, 5, 65 , 0.7)"#"rgb(226,141,164)"
}

cmap_great_fun = {
    "GlyX3":"rgba( 203, 83, 83 , 1)",
    "CoBaHMA_X1":"rgba(172, 221, 146,0.7)",
    "CoBaHMA_X2":"rgba(113, 197, 68,0.7)",
    "CoBaHMA_X3": "rgba(69, 125, 38,0.7)",    
    "P-type":"rgba(0,255,255,0.7)",
    "Phosphatase":"rgba(255,105,180,0.7)",
    "ABC transporter":"rgba(255,165,0,0.7)"
}



In [259]:
for cmap in [cmap_taxo_rgba_no_opa,cmap_taxo_rgba]:
    sankeydf["color"] = sankeydf.apply(lambda x: cmap_great_fun[x.src] if x.src in cmap_great_fun else cmap[x.target],axis=1 )
#    sankeydf["color_r"] = sankeydf.apply(lambda x: cmap_great_fun[x.src] if x.src in cmap_great_fun else cmap_taxo_rgba_no_opa[x.target],axis=1 )
    sankeydf
    sankey_idx_map_r = {}
    for i,j in sankey_idx_map.items():
        if i in cobahma_only_commu:
            i = "<b>{}*</b>".format(i)
        sankey_idx_map_r[i]=j    
    
    
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = list(sankey_idx_map_r.keys()),
          color = "grey"#[sankey_cmap[i] if i in sankey_cmap else "grey"  for i in list(sankey_idx_map.keys()) ]
        ),
        link = dict(
          source = list(sankeydf.src_r), # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = list(sankeydf.target_r),
          value = list(sankeydf.value),
          color = sankeydf.color,

      ))])
    basename = 'Supplementary_data_5_cyano_highlight'
    if cmap['Cyanobacteria'] == 'rgba(15, 133, 84, 0.7)':
        basename = 'Supplementary_data_5'        
    
    fig.update_layout(font_size=10,height=1000)
    fig.write_html(FIGURESDIR + "/" + basename + ".html")
    fig.write_image(FIGURESDIR + "/" + basename + ".pdf")
    fig.write_image(FIGURESDIR + "/" + basename + ".svg")
    fig.show()

# Check taxonomy for small community and singletons

In [ ]:
small_commu_nodes = []
for _ , sequences in communities.items():
    if len(sequences)<5:
        small_commu_nodes+=sequences
            
taxodf[taxodf.index.isin(small_commu_nodes)].groupby(["k","p"]).count() 

# SPLIT NETWORK BY GREAT FUNCTION GROUP

### Group communityies based on great function group
- ABC
- phosphate
- P-type
- Single CoBaHMA
- ccyA
- 2x CoBaHMA
- 3x CoBaHMA

In [262]:
communities_desc= pd.read_csv(NETWORKDIR + "/community_desc.tsv",sep="\t",header=0,index_col=0)
communities_desc.head()

,Community Rep,Rep degree,Community Size,subgraph,Rep Degree,mean_seq_len,longest_seqs,longest_len,shortest_seqs,shortest_len,prefix,_commu,community_track,IPRS,IPRS_Functions,DomainMapper_Ecod,DomainMapper_Functions
Community,,,,,,,,,,,,,,,,,
C3,UniRef100_A0A1M3N6F4,0,1,Subgraph_178,0,163.000000,UniRef100_A0A1M3N6F4,163,UniRef100_A0A1M3N6F4,163,us,3,us_C3,NaN,NaN,NaN,NaN
C4,UniRef100_A0A1H0C0W7,12,18,Subgraph_17,12,194.111111,UniRef100_A0A6P1HK34,202,UniRef100_A0A0A5GGF7;UniRef100_A0A1L3MRH9;UniR...,190,ubc,4,ubc_C4,NaN,NaN,NaN,NaN
C5,UniRef100_A0A662ZLY5,4,5,Subgraph_52,4,218.800000,UniRef100_R5EHL9,222,UniRef100_A0A662ZMY2,210,ubc,5,ubc_C5,NaN,NaN,NaN,NaN
C6,UniRef100_A0A0R1VKN6,0,1,Subgraph_179,0,487.000000,UniRef100_A0A0R1VKN6,487,UniRef100_A0A0R1VKN6,487,us,6,us_C6,IPR001757;IPR008250;IPR023214;IPR023299,"P-type ATPase;P-type ATPase, A domain superfam...",5073.1.2.3;10.13.1.2;267.1.1.5,E1-E2_ATPase_C;E1-E2_ATPase_N;P-type_ATPase_Cu...
C7,UniRef100_A0A4Q5L4V0,4,5,Subgraph_53,4,795.800000,UniRef100_A0A5Q0BFQ5,821,UniRef100_A0A858QAA3,786,sc,7,sc_C7,IPR008250;IPR023299;IPR023214;IPR036412,"P-type ATPase, A domain superfamily;P-type ATP...",10.13.1.2;2006.1.1.42,E1-E2_ATPase_N;Hydrolase_3_1


In [272]:
great_func_seq_df = pd.read_csv(
    "../datas/sequence_great_func_group.tsv",sep="\t",header=None
)

great_func_seq_df.columns=["seqid","great_func_group","cid"]
great_func_seq_df['cid']  = great_func_seq_df.cid.map(community_label_map_dict)
great_func_seq_df["is_singleton"] = great_func_seq_df.apply(lambda x: True if len(communities[x.cid]) < 2 else False, axis=1 )
great_func_seq_df


,seqid,great_func_group,cid,is_singleton
0,CP006735.1_prot_AHB89148.1_1755,GlyX3,C724,False
1,Candidatus_Synechococcus_calcipolaris,GlyX3,C192,False
2,Chroococcidiopsis_thermalis_PCC_7203,GlyX3,C192,False
3,Cyanothece_sp_PCC_7425,GlyX3,C239,False
4,MWMJ01000016.1_prot_OUE49294.1_331,GlyX3,C700,True
...,...,...,...,...
2222,UniRef100_X8BLG9,CoBaHMA_X1,C71,False
2223,UniRef100_X8CAT3,CoBaHMA_X1,C383,False
2224,UniRef100_X8CK65,CoBaHMA_X1,C422,True
2225,UniRef100_X8HQE8,CoBaHMA_X1,C141,False


In [275]:

gfg = ['P-type', 'ABC transporter', 'Phosphatase', 'GlyX3', 'CoBaHMA_X3',  'CoBaHMA_X2',
    'CoBaHMA_X1']


list_community = great_func_seq_df.cid.unique()
great_fun_commu = {}
for f in gfg:
    coms = great_func_seq_df[great_func_seq_df.great_func_group==f].cid.unique()
    great_fun_commu[f]=coms
    #great_func_seq_df = great_func_seq_df[~great_func_seq_df.cid.isin(coms)]
great_fun_commu    

{'P-type': array(['C525', 'C744', 'C366', 'C710', 'C170', 'C289', 'C486', 'C536',
        'C191', 'C720', 'C500', 'C98', 'C20', 'C385', 'C8', 'C6', 'C547',
        'C470', 'C429', 'C94', 'C556', 'C657', 'C550', 'C712', 'C658',
        'C579', 'C468', 'C420', 'C11', 'C582', 'C293', 'C281', 'C503',
        'C77', 'C57', 'C614', 'C612', 'C162', 'C42', 'C413', 'C140',
        'C692', 'C183', 'C224', 'C697', 'C683', 'C703', 'C439', 'C250',
        'C729', 'C689', 'C694', 'C656', 'C386', 'C670', 'C40', 'C295',
        'C739', 'C180', 'C244', 'C242', 'C510', 'C598', 'C46', 'C84',
        'C290', 'C53', 'C154', 'C390', 'C223', 'C647', 'C514', 'C7', 'C91',
        'C616', 'C681', 'C447', 'C625', 'C395', 'C613', 'C669', 'C594',
        'C34', 'C2', 'C219', 'C564', 'C128', 'C691', 'C43', 'C590', 'C541',
        'C438', 'C526', 'C576', 'C615', 'C179', 'C488', 'C1', 'C103',
        'C288', 'C287', 'C286', 'C480', 'C271', 'C89', 'C110', 'C74',
        'C716', 'C464', 'C72', 'C610', 'C146', 'C375', '

In [276]:
great_func_reformat = {}
for _ , e in great_func_seq_df[great_func_seq_df.is_singleton==False].iterrows():
    seqid = e.seqid
    if e.great_func_group not in great_func_reformat:
        great_func_reformat[e.great_func_group] = []
    great_func_reformat[e.great_func_group].append(seqid)
great_func_reformat.keys()

dict_keys(['GlyX3', 'CoBaHMA_X1', 'P-type', 'ABC transporter', 'CoBaHMA_X2', 'Phosphatase', 'CoBaHMA_X3'])

In [278]:
from scipy.stats import sem

#for func, sequences in great_func_reformat.items():
#os.makedirs("../network_clean/network_by_great_func",exist_ok=True)
singletons_great_fun = {}
with open(NETWORKDIR+"/commu_mean_seq_len_map_file.tsv" , 'w' ) as f:
    f.write("Community\tgreat func\tmean length\tsd length\tsem length\tcommu size\n")
    for fun,cids in great_fun_commu.items():
        seq = []
        singletons_great_fun[fun] = []
        for cid in cids:
            #if len(communities[cid]) >= 2:
            seq += communities[cid]
            seq_len  = [len(records[s].seq) for s in communities[cid]]
            mean_len = round(np.mean(seq_len),1)
            sem_len  = round(sem(seq_len),1) if len(seq_len) >1 else 0
            sd_len = round(np.std(seq_len),1)
            f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(
                cid,
                fun,
                mean_len,
                sd_len,
                sem_len,                
                len(communities[cid])
                #commu_label_for_cyto)
            ))                
                
            #else:
            #    singletons_great_fun[fun].append((cid,records[communities[cid][0]]))
        if seq:
            pass
            #nwx.write_gexf(nwx.subgraph(G,seq), "../network_clean/network_by_great_func/{fun}/{fun}.gexf".format(fun=fun))
            

# Plot distribution

In [346]:
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
f = NETWORKDIR+"/commu_mean_seq_len_map_file.tsv"
df = pd.read_csv(f,sep='\t',header=0,index_col=0)
cmap_great_fun = {
    "GlyX3":(231, 29, 54),
    "CoBaHMA_X1": (117, 227, 125),
    "CoBaHMA_X2":(57, 148, 50),
    "CoBaHMA_X3": (57, 83, 50),
    "P-type":(65, 234, 212),
    "Phosphatase":(231, 143, 142),
    "ABC transporter":(255, 127, 17)
}

count_by_commu = {} 
for f in df["great func"].unique():
    tmpdf = df[df["great func"]==f]
    nsingle=tmpdf[(tmpdf['commu size']==1)].shape[0]
    ncommu= tmpdf[(tmpdf['great func']==f)].shape[0]-nsingle
    
    count_by_commu[f]={"commu":ncommu,"single":nsingle}


colors = {i:"rgb{}".format(str(j)) for i,j in cmap_great_fun.items()}

d0= px.scatter(
        df[df["commu size"]>1].reset_index(),
        y='Community',x='mean length',
        #color_discrete_sequence=['rgba(0,0,0,0.5)'],
        error_x = 'sd length',
        text='Community', 
        color='great func',
        color_discrete_map=colors,
    )

d0.update_traces(
        textposition='top center',textfont_size=7.5
    )
d0.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',   
    yaxis={ 
        "title": "", 
           "showticklabels":False,
           "showline":True, 
           "linewidth":1, 
           "linecolor":'lightgrey', 
           "tickfont":{"size":6},
           #"gridwidth":1, "gridcolor":'lightgrey'
           #"linewidth":1, 
           #"linecolor":'black',
          },
    xaxis={ 
    "showline":True, 
    "linewidth":1, 
    "linecolor":'lightgrey', 
    #"gridwidth":1, "gridcolor":'lightgrey'        
    },
    height=800,
)

In [351]:
def make_distplot_traces(df,func):
    cmap_great_fun = {
        "GlyX3":(231, 29, 54),
        "CoBaHMA_X1": (117, 227, 125),
        "CoBaHMA_X2":(57, 148, 50),
        "CoBaHMA_X3": (57, 83, 50),
        "P-type":(65, 234, 212),
        "Phosphatase":(231, 143, 142),
        "ABC transporter":(255, 127, 17)
    }

    df["is_singleton"] = df.apply(lambda x : True if x["commu size"]==1 else False,axis=1)
    df = df[df["great func"].isin(func)]

    count_by_commu = {}
    hist_data = []
    group_labels = [] # name of the dataset
    colors = []
    for f in func: 
        tmpdf = df[(df['great func']==f)]
        nsingle=tmpdf[(tmpdf['commu size']==1)].shape[0]
        ncommu= tmpdf[(tmpdf['great func']==f)].shape[0]-nsingle

        count_by_commu[f]={"commu":ncommu,"single":nsingle}

        # make distri chart

        commu = tmpdf[tmpdf['commu size']!=1]['mean length']
        single = tmpdf[tmpdf['commu size']==1]['mean length']
        col = str(cmap_great_fun[f])
        print(col)
        if commu.shape[0]>1:
            hist_data.append(commu)
            group_labels.append('{} [communities]'.format(f))
            opacity = 1
            commucol = col.replace(")",",{})".format(opacity))
            colors.append("rgba{}".format(commucol))
        if single.shape[0]>1:
            hist_data.append(single)
            group_labels.append("{} [singletons]".format(f))
            opacity = 1
            singlecol = col.replace(")",",{})".format(opacity))
            colors.append("rgba{}".format(singlecol))
            


    dist = ff.create_distplot(
        hist_data, 
        group_labels,
        bin_size=[25 for i in range(len(colors))],
        colors=colors,
        show_hist=False,

    )
    
    dist.for_each_trace(lambda t: t.update(line={"dash":"dash"}) if t.mode=="lines" and "singleton" in t.legendgroup else t)
    
    dist.update_layout(
        #paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',    
        xaxis={ 
               "showline":True, 
               "linewidth":1, 
               "linecolor":'black',
              },
        yaxis={ 
            "title": "Frequency",
           "showline":True, 
            "linewidth":1, 
            "linecolor":'black',
        }
    )
    
    
    return dist


In [350]:
d1 = make_distplot_traces(df,['CoBaHMA_X1'])
d2 = make_distplot_traces(df,['P-type'])
d3 = make_distplot_traces(df,['Phosphatase','ABC transporter','CoBaHMA_X2','CoBaHMA_X3','GlyX3'])

(117, 227, 125)
(65, 234, 212)
(231, 143, 142)
(255, 127, 17)
(57, 148, 50)
(57, 83, 50)
(231, 29, 54)


In [363]:
f = make_subplots(rows=2, cols=1,
              shared_xaxes=True,
              row_heights=[0.6, 0.4],# 0.1, 0.025, 0.2],
              vertical_spacing=0.005,
              x_title='Mean sequence length',                 
             )

#for i in d0.data:
#    f.add_trace(i,row=1,col=1)

for i in d1._data_objs:
    f.add_trace(i,row=1,col=1)
    f.append_trace(d1['data'][2], 1, 1)
    f.append_trace(d1['data'][3], 1, 1) 
    
    
for i in d2._data_objs:
    f.add_trace(i,row=1,col=1)
    f.append_trace(d2['data'][2], 1, 1)
    f.append_trace(d2['data'][3], 1, 1)    
    
for i in d3._data_objs:
    #f.add_trace(i,row=6,col=1)  
    for t in d3['data']:
        if t.marker['symbol'] == 'line-ns-open':
            f.append_trace(t, 2, 1)
            
    
f.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',    
    height=1800,
    width=1250
    #font=dict(
        #family="Courier New, monospace",
        #size=10,  # Set the font size here
        #color="RebeccaPurple"
    #    )
)

f.update_yaxes(title_text="Community",showticklabels=False,linewidth=1,linecolor='lightgrey', row=1, col=1)
f.update_yaxes(title_text="Probability density",linewidth=1,linecolor='lightgrey', row=2, col=1)
f.update_yaxes(linewidth=1,linecolor='lightgrey', row=3, col=1)
f.update_xaxes(linewidth=1,linecolor='lightgrey', row=3, col=1)
#f.update_layout(showlegend=False)

text = ""
for func,values in count_by_commu.items():
    text += "<b>{}:</b><br>-communities: {}<br>-singletons: {}<br>".format(func,values["commu"],values["single"])

    
f.add_annotation(xref="paper", yref="paper",x=1.3, y=0.2,
            text= text,    
            showarrow=False,
        )
f.update_annotations(align="left")
f.update_layout(height=800)
f.write_image(FIGURESDIR+ "/S6_side.svg")
f.write_image(FIGURESDIR+ "/S6_side.pdf")
f.show()

# Create subgraph

In [280]:
great_fun_iprs_df  = pd.read_csv('../datas/great_fun_IPR.tsv',sep="\t")
great_fun_iprs_df["IPR accession"] = great_fun_iprs_df["IPR accession"].apply(lambda x: x.split()[0])
great_fun_iprs_df.group = great_fun_iprs_df.group.apply(lambda x:x.replace("Cation-transporting ",""))

print(great_fun_iprs_df.group.unique())
great_fun_iprs_df = great_fun_iprs_df[great_fun_iprs_df.group.isin(['P-type ATPase' , 'Phosphatase', 'ABC transporter'])]
great_fun_iprs_map =  great_fun_iprs_df.set_index('IPR accession').group.to_dict()
great_fun_iprs_df.head()


['P-type ATPase' 'HMA' 'Phosphatase' 'ABC transporter' 'Hydrolase']


,IPR accession,Annotation,group
0,IPR006068,"Cation-transporting P-type ATPase, C-terminal",P-type ATPase
1,IPR004014,"Cation-transporting P-type ATPase, N-terminal",P-type ATPase
2,IPR023298,"P-type ATPase, transmembrane domain superfamily",P-type ATPase
3,IPR027256,"P-type ATPase, subfamily IB",P-type ATPase
4,IPR023299,"P-type ATPase, cytoplasmic domain N",P-type ATPase


In [281]:
for i in annot_df[annot_df.Desc=="GlyX3"].Community.unique():
    print(i,len(communities[i]))

C682 3
C192 15
C724 2
C239 4
C119 1
C254 4
C109 2
C700 1


In [310]:
#
community_labels = list(communities.keys())
d = {i:[] for i in great_fun_iprs_map.values()}
for i,j in communities.items():
    iprs = community_desc_table.loc[i,'IPRS'].split(';')
    great_fun = []
    for ipr in iprs:
        if ipr in great_fun_iprs_map:
            if i not in d[great_fun_iprs_map[ipr]]:
                d[great_fun_iprs_map[ipr]].append(i)
            if great_fun_iprs_map[ipr] not in great_fun:
                great_fun.append(great_fun_iprs_map[ipr])
    if len(great_fun) ==1:
        community_labels.remove(i)
len(communities)-len(community_labels)           

162

In [312]:
tmp = annot_df[annot_df.Community.isin(community_labels)]

cobahma_x3 = tmp[tmp.Desc_copy=='CoBaHMA_2'].Community.unique().tolist()
cobahma_x2 = tmp[(tmp.Desc_copy=='CoBaHMA_1') & (~tmp.Community.isin(cobahma_x3))].Community.unique().tolist()
cobahma_x1 = tmp[(tmp.Desc_copy=='CoBaHMA_0') & (~tmp.Community.isin(cobahma_x3+cobahma_x2))].Community.unique().tolist()

d["CoBaHMA_X3"] = cobahma_x3
d["CoBaHMA_X2"] = cobahma_x2
d["CoBaHMA_X1"] = cobahma_x1

community_great_group_map = {}
for i,j in d.items():
    for c in j:
        community_great_group_map[c] = i
        
pd.DataFrame({'Group':community_great_group_map}).to_csv('../datas/communities_and_great_fun_group_map.tsv',sep='\t')       
community_great_group_map


{'C6': 'P-type ATPase',
 'C7': 'P-type ATPase',
 'C8': 'P-type ATPase',
 'C28': 'P-type ATPase',
 'C34': 'P-type ATPase',
 'C429': 'P-type ATPase',
 'C40': 'P-type ATPase',
 'C43': 'P-type ATPase',
 'C84': 'P-type ATPase',
 'C46': 'P-type ATPase',
 'C52': 'P-type ATPase',
 'C53': 'P-type ATPase',
 'C57': 'P-type ATPase',
 'C74': 'P-type ATPase',
 'C82': 'P-type ATPase',
 'C89': 'P-type ATPase',
 'C94': 'P-type ATPase',
 'C525': 'P-type ATPase',
 'C99': 'P-type ATPase',
 'C103': 'P-type ATPase',
 'C110': 'P-type ATPase',
 'C115': 'P-type ATPase',
 'C0': 'P-type ATPase',
 'C468': 'P-type ATPase',
 'C128': 'P-type ATPase',
 'C129': 'P-type ATPase',
 'C510': 'P-type ATPase',
 'C140': 'P-type ATPase',
 'C146': 'P-type ATPase',
 'C568': 'P-type ATPase',
 'C413': 'P-type ATPase',
 'C160': 'P-type ATPase',
 'C162': 'P-type ATPase',
 'C169': 'P-type ATPase',
 'C170': 'P-type ATPase',
 'C175': 'P-type ATPase',
 'C179': 'P-type ATPase',
 'C180': 'P-type ATPase',
 'C181': 'P-type ATPase',
 'C183':

In [313]:
import networkx as nx
graph = nx.read_gexf(NETWORKDIR + "/network.clean.gexf")
os.makedirs(NETWORKDIR + "/networks_by_fun",exist_ok=True)
for func, commus in d.items():
    nodes = []
    for commu in commus:
        nodes += communities[commu]
    sgraph = graph.subgraph(nodes)
    nx.write_gexf(sgraph,NETWORKDIR + "/networks_by_fun/{}.gexf".format(func))


In [338]:
template = """
<svg xmlns="http://www.w3.org/2000/svg"
     xmlns:xlink="http://www.w3.org/1999/xlink">
  <text font-size="0.75em" font-family="Arial, Helvetica, sans-serif">
      <tspan dy="1em" x="10">{community} (N={size})</tspan>
      <tspan dy="1em" x="10">L={mean_len}±{std}</tspan>  
  </text>
</svg>
"""


singletons_datas = []
for func, commus in d.items():
    #svg = open(NETWORKDIR + "/networks_by_fun/{}.gexf")
    os.makedirs(NETWORKDIR+'/Supplementary_fig_6/Commu_text_annot_label_corrected/'+func,exist_ok=True)
    for cl in commus:
        #commu_label = "_".join(cl.split('_')[1:])
        l = [length_dict[i] for i in communities[cl]]
        commu_size = len(l)
        if commu_size > 1:
            mean_length = round(np.mean(l),1)
            std_length  = round(np.std(l),1)                        
            svg = template.format(community = cl, size=commu_size,
                           mean_len = mean_length, std = std_length)
            f = open(NETWORKDIR+'/Supplementary_fig_6/Commu_text_annot_label_corrected/{}/{}.svg'.format(func,commu_label),'w')
            f.write(svg)
            f.close()
        else:
            singletons_datas.append((func, cl  , length_dict[communities[cl][0]] ))
singletons_df = pd.DataFrame(singletons_datas)
singletons_df[0].unique()#.groupby(1).count()

array(['P-type ATPase', 'ABC transporter', 'CoBaHMA_X3', 'CoBaHMA_X2',
       'CoBaHMA_X1'], dtype=object)

{'CP006735.1_prot_AHB89148.1_1755': 'ccyA+',
 'Candidatus_Synechococcus_calcipolaris': 'ccyA+',
 'Chroococcidiopsis_thermalis_PCC_7203': 'ccyA+',
 'Cyanothece_sp_PCC_7425': 'ccyA+',
 'MWMJ01000016.1_prot_OUE49294.1_331': 'ccyA+',
 'NZ_ADXL01000065.1_prot_WP_010309903.1_1141': 'ccyA+',
 'NZ_AP018202.1_prot_WP_126984835.1_8': 'ccyA+',
 'NZ_CH724158.1_prot_WP_007100847.1_828': 'ccyA+',
 'NZ_CP018092.1_prot_WP_099799411.1_2103': 'ccyA+',
 'NZ_JJML01000036.1_prot_WP_052128762.1_2284': 'ccyA+',
 'Synechococcus_PCC_6716': 'ccyA+',
 'Synechococcus_PCC_6717': 'ccyA+',
 'Synechococcus_sp_PCC_6312': 'ccyA+',
 'Thermosynechococcus_elongatus': 'ccyA+',
 'UniRef100_A0A098TJ90': 'ccyA+',
 'UniRef100_A0A2D2Q3J4': 'ccyA+',
 'UniRef100_A0A3M2FQJ0': 'ccyA+',
 'UniRef100_A0A433VEI5': 'ccyA+',
 'UniRef100_A0A5C2M6G8': 'ccyA+',
 'UniRef100_A3Z4P0': 'ccyA+',
 'UniRef100_B8HLW5': 'ccyA+',
 'UniRef100_K9S008': 'ccyA+',
 'UniRef100_Q8DMV2': 'ccyA+',
 'UniRef100_V5V6W9': 'ccyA+',
 'WP_052289956.1': 'ccyA+'}

In [388]:
dfs = []
for i in os.listdir('../datas/blast_results/'):
    d = pd.read_csv("../datas/blast_results/" + i , sep='\t',header=None)
    d['proteom'] = i
    dfs.append(d)
blast_df = pd.concat(dfs,axis=0)
blast_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,proteom
0,UniRef100_A0A1H0C0W7,CAIVJA010000041.1_4,42.000,26,2,11,58,5,53,0.010,33.5,194,683,50,GCA_903906125.1.tsv
1,UniRef100_A0A1H0C0W7,CAIVJA010000067.1_9,35.714,31,3,7,57,11,66,0.026,31.2,194,137,56,GCA_903906125.1.tsv
2,UniRef100_A0A1H0C0W7,CAIVJA010000283.1_2,21.250,55,1,33,104,276,355,0.110,30.4,194,410,80,GCA_903906125.1.tsv
3,UniRef100_A0A1H0C0W7,CAIVJA010000227.1_1,33.333,41,3,38,109,119,183,0.140,30.0,194,618,72,GCA_903906125.1.tsv
4,UniRef100_A0A1H0C0W7,CAIVJA010000335.1_2,20.690,45,1,12,68,19,76,0.150,29.3,194,189,58,GCA_903906125.1.tsv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,UniRef100_A0A351WUV5,lcl|NZ_JADEVX010000171.1_cds_WP_036399255.1_1298,28.333,41,1,28,87,71,128,2.200,25.0,132,1035,60,GCF_015207925.1.tsv
1221,UniRef100_A0A351WUV5,lcl|NZ_JADEVX010000026.1_cds_WP_002740841.1_2154,29.508,38,1,38,93,127,187,2.300,25.0,132,435,61,GCF_015207925.1.tsv
1222,UniRef100_A0A351WUV5,lcl|NZ_JADEVX010000087.1_cds_WP_002741762.1_4438,50.000,9,0,71,88,16,33,5.300,23.1,132,87,18,GCF_015207925.1.tsv
1223,UniRef100_A0A351WUV5,lcl|NZ_JADEVX010000018.1_cds_WP_002745227.1_1412,26.829,30,0,35,75,700,740,9.600,23.1,132,857,41,GCF_015207925.1.tsv


In [389]:
blast_df.columns = "qseqid sseqid pident mismatch gapopen qstart qend sstart send evalue bitscore qlen slen length".split() + ["proteom"]
blast_df['qcov'] = (blast_df['qend'] - blast_df['qstart']) / blast_df['qlen']
blast_df['scov'] = (blast_df['send'] - blast_df['sstart']) / blast_df['slen']

In [390]:
blast_df.head()

,qseqid,sseqid,pident,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,length,proteom,qcov,scov
0,UniRef100_A0A1H0C0W7,CAIVJA010000041.1_4,42.000,26,2,11,58,5,53,0.010,33.5,194,683,50,GCA_903906125.1.tsv,0.242268,0.070278
1,UniRef100_A0A1H0C0W7,CAIVJA010000067.1_9,35.714,31,3,7,57,11,66,0.026,31.2,194,137,56,GCA_903906125.1.tsv,0.257732,0.401460
2,UniRef100_A0A1H0C0W7,CAIVJA010000283.1_2,21.250,55,1,33,104,276,355,0.110,30.4,194,410,80,GCA_903906125.1.tsv,0.365979,0.192683
3,UniRef100_A0A1H0C0W7,CAIVJA010000227.1_1,33.333,41,3,38,109,119,183,0.140,30.0,194,618,72,GCA_903906125.1.tsv,0.365979,0.103560
4,UniRef100_A0A1H0C0W7,CAIVJA010000335.1_2,20.690,45,1,12,68,19,76,0.150,29.3,194,189,58,GCA_903906125.1.tsv,0.288660,0.301587


In [403]:
p = blast_df[(blast_df.qcov >= 0.80) & (blast_df.scov >= 0.80) & (blast_df.pident >= 50 )  ].proteom.unique().tolist()
[print(i.replace(".tsv","")) for i in p]



GCA_903906125.1
GCF_019264515.1
GCF_015296045.1
GCA_014946735.1
GCA_903931035.1
GCA_003261315.1
GCA_903908235.1
GCF_014654725.1
GCA_025999015.1
GCA_003015105.1
GCA_903901165.1
GCA_007095845.1
GCA_000317285.1
GCF_014697535.1
GCA_003242955.1
GCA_022570815.1
GCA_028328965.1
GCA_000317125.1
GCA_000775285.1
GCA_007095695.1
GCA_903898235.1
GCA_913061215.1
GCF_000317265.1
GCA_000317265.1
GCF_000775285.1
GCF_000317125.1
GCA_007095795.1
GCF_028328965.1
GCA_903823855.1
GCA_014697535.1
GCF_000317285.1
GCA_009996485.1
GCA_903894585.1
GCF_003015105.1
GCA_014654725.1
GCA_015296045.1
GCA_019264515.1
GCA_903835705.1
GCA_003555505.2
GCF_001990805.1
GCF_000317205.1
GCA_016106025.1
GCA_903882195.1
GCA_003991895.1
GCA_903929635.1
GCF_029582805.1
GCF_000179235.1
GCA_003555505.1
GCF_001990805.2
GCF_002870615.1
GCA_903912685.1
GCF_003990665.1
GCF_025999095.1
GCF_019264575.1
GCA_024239075.1
GCA_019104765.1
GCA_025454425.1
GCF_001990805.3
GCA_026222535.1
GCF_001548455.1
GCF_000179235.2
GCA_000179235.2
GCA_0226

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [395]:
commu_with_match_ccya_protem = blast_df[(blast_df.qcov >= 0.80) & (blast_df.scov >= 0.80) & (blast_df.pident >= 50 )  ].qseqid.unique().tolist()

community_table[community_table['Community Rep'].isin(commu_with_match_ccya_protem)].index



,Community Rep,Rep degree,Community Size,subgraph,Rep Degree,mean_seq_len,longest_seqs,longest_len,shortest_seqs,shortest_len,prefix,_commu,community_track
Community,,,,,,,,,,,,,
C525,UniRef100_A0A5C7KBI4,36,61,Subgraph_3,36,805.508197,UniRef100_A0A522WSV7,824,UniRef100_A0A1L6LBI1,776,bc,525,bc_C525
C140,UniRef100_B8ETC0,10,19,Subgraph_16,10,858.421053,UniRef100_A0A432MEJ3;UniRef100_A0A3L7S6G8,878,UniRef100_Q3AI40,829,bc,140,bc_C140
C192,UniRef100_Q8DMV2,11,15,Subgraph_20,11,331.600000,Candidatus_Synechococcus_calcipolaris,362,UniRef100_A0A3M2FQJ0;UniRef100_A0A5C2M6G8;Ther...,320,bc,192,bc_C192
C283,UniRef100_B1WT30,23,31,Subgraph_7,23,189.580645,UniRef100_A0A1C0VLK1,206,UniRef100_A0A844GW67,176,ubc,283,ubc_C283
C331,UniRef100_Q8YVH2,12,17,Subgraph_18,12,383.352941,UniRef100_A0A1Q4RP66,406,UniRef100_A0A1D2P5I6,359,bc,331,bc_C331
C341,UniRef100_A0A6M0J010,18,23,Subgraph_7,18,207.695652,UniRef100_A0A140K985,223,UniRef100_A0A2H2XPK7,181,bc,341,bc_C341
C393,UniRef100_A0A1Z4FYZ0,43,62,Subgraph_2,43,186.951613,UniRef100_A0A1U7HNL3,208,UniRef100_A0A2U3JXY9,168,ubc,393,ubc_C393
C668,UniRef100_A0A0C2QKB5,7,8,Subgraph_39,7,421.750000,UniRef100_A0A1Z4NNN6,451,UniRef100_A0A367QF88,411,s,668,s_C668
C98,UniRef100_A0A0M0SKF3,12,13,Subgraph_22,12,1063.769231,UniRef100_A0A0M0SKF3,1092,UniRef100_A0A0S8G388,1048,s,98,s_C98


In [399]:
print("\n".join(community_table[community_table['Community Rep'].isin(commu_with_match_ccya_protem)].index))

C525
C140
C192
C283
C331
C341
C393
C668
C98
C550
C419
C20
C623
C656
C643
C654
